## Load data

In [1]:
import pandas as pd

In [2]:
df_base = pd.read_csv('../data/general/pjm_pivot.csv', index_col=0, parse_dates=True)
df_base.head()

,AE,AEP,AP,ATSI,BC,CE,DAY,DEOK,DOM,DPL,DUQ,EKPC,JC,ME,PE,PEP,PL,PN,PS,RECO
period,,,,,,,,,,,,,,,,,,,,
2018-07-01 05:00:00,1301,12187,4384,6960,3134,12136,1619,2895,9775,1919,1466,1209,2388,1442,4397,3165,3835,1611,5009,186
2018-07-01 06:00:00,1314,11946,4391,6762,3139,11872,1605,2857,9787,1950,1455,1197,2416,1465,4423,3156,3901,1641,4990,187
2018-07-01 07:00:00,1410,12664,4757,6670,3377,11992,1707,2997,10453,2160,1528,1273,2644,1605,4743,3332,4232,1728,5267,202
2018-07-01 08:00:00,1567,14069,5308,7065,3788,12860,1916,3331,11734,2470,1688,1472,3064,1784,5230,3679,4613,1899,5735,230
2018-07-01 09:00:00,1749,15610,5862,7833,4262,14212,2145,3703,13084,2765,1875,1656,3569,1972,5752,4085,5014,2055,6299,259


## Select region

In [3]:
region = 'AE'

In [4]:
# Parameters
region = "DUQ"


In [5]:
df = df_base.loc[:, [region]]
df

,DUQ
period,
2018-07-01 05:00:00,1466
2018-07-01 06:00:00,1455
2018-07-01 07:00:00,1528
2018-07-01 08:00:00,1688
2018-07-01 09:00:00,1875
...,...
2023-11-01 00:00:00,1449
2023-11-01 01:00:00,1461
2023-11-01 02:00:00,1431


## Data Preprocessing

### Train Test Split

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, shuffle=False)
test, prod = train_test_split(test, test_size=0.5, shuffle=False)

### Export Data

In [7]:
import os

path_region = f'../data/regions/{region}'

if not os.path.exists(path_region):
    os.makedirs(path_region)

for df, name in zip([train, test, prod], ['train', 'test', 'prod']):
    df.to_csv(f'{path_region}/{name}.csv')

### Scale Data

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))

train_norm = scaler.fit_transform(train)
test_norm = scaler.transform(test)

### Create Sequences

In [9]:
import numpy as np

def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 24 # Use 24 hours prior to predict the following hour
X_train, y_train = create_sequences(train_norm, sequence_length)
X_test, y_test = create_sequences(test_norm, sequence_length)

## Modelling

### Design NN Architecture

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\jesus\miniconda3\envs\tf-base\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\jesus\miniconda3\envs\tf-base\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\jesus\miniconda3\envs\tf-base\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\jesus\miniconda3\envs\tf-base\lib\site-packages\tensorf

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


c:\Users\jesus\miniconda3\envs\tf-base\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\jesus\miniconda3\envs\tf-base\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\jesus\miniconda3\envs\tf-base\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\jesus\miniconda3\envs\tf-base\lib\

### Train Model

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    X_train, y_train, epochs=50, batch_size=64, verbose=0,
    validation_data=(X_test, y_test), callbacks=[early_stop])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Calculate Predictions

In [12]:
y_pred = model.predict(X_test)
y_pred

array([[0.13401301],
       [0.14411716],
       [0.13959089],
       ...,
       [0.26376995],
       [0.29595473],
       [0.3236659 ]], dtype=float32)

### Comparison: Real Data & Prediction

#### Descale Data

In [13]:
y_pred_scaled_inverse = scaler.inverse_transform(y_pred)
y_true = scaler.inverse_transform(y_test)

#### Create DataFrame

In [14]:
dic_pred = {
    'y_pred': y_pred.flatten(),
    'y_pred_scaled_inverse': y_pred_scaled_inverse.flatten(),
    'y_true': y_true.flatten(),
}

df_pred = pd.DataFrame(dic_pred)
df_pred

,y_pred,y_pred_scaled_inverse,y_true
0,0.134013,1196.223389,1192.0
1,0.144117,1214.410889,1187.0
2,0.139591,1206.263672,1224.0
3,0.168039,1257.470337,1294.0
4,0.204404,1322.927002,1307.0
...,...,...,...
6707,0.227596,1364.672852,1368.0
6708,0.234372,1376.869019,1394.0
6709,0.263770,1429.786011,1438.0
6710,0.295955,1487.718506,1488.0


#### Evaluate Model

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
mean_squared_error(df_pred.y_true, df_pred.y_pred_scaled_inverse, squared=False)

25.365368672013346

In [17]:
model.save(f'../models/{region}.keras')